In [ ]:
# pip install pandas numpy matplotlib seaborn openpyxl jupyter

In [10]:
# 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

df = pd.read_csv('./유동인구_매출_비교분석_2022_2025.csv', encoding='utf-8')

print(df)
print(df.shape)           # 행, 열 개수
print(df.info())          # 컬럼 정보
print(df.describe())      # 기술통계



# 시각화
fig = px.histogram(df, x='평균_유동인구', y='행정동명', title='유동인구 분포', 
                   labels={'평균_유동인구': '평균 유동인구'},
                   template='plotly_white')
fig.show()

        행정동코드    행정동명  유동인구_구간   평균_유동인구  편의점_수    평균_매출(원)  편의점당_매출(원)  \
0    11680750     수서동  1_하위20%   2236113     14  8699190278   621370734   
1    11710642    문정2동  1_하위20%   2826795     62  8332065322   134388150   
2    11680720    일원본동  1_하위20%   2480840     14  6533498203   466678443   
3    11140520     소공동  1_하위20%   2709080     49  6392294656   130454993   
4    11170650   이태원1동  1_하위20%   2595249     20  6196627455   309831373   
..        ...     ...      ...       ...    ...         ...         ...   
414  11350665  상계3·4동  5_상위20%   8372870     17  1049787837    61752226   
415  11500593    화곡8동  5_상위20%   8339626     16  1029230316    64326895   
416  11350665  상계3?4동  5_상위20%   8372870     17   982020782    57765928   
417  11290705     종암동  5_상위20%  10904975     27   925978490    34295500   
418  11305534     삼양동  5_상위20%  11316800     10   590602037    59060204   

     주말_매출_비율(%)  여성_매출_비율(%)  20대_매출_비율(%)  30대_매출_비율(%)  60대+_매출_비율(%)  \
0           25.4       